Malarial_cell_classification_using_CNN

Install tensorflow and keras

In [52]:
# import libraries
import os
import cv2
import keras
import numpy as np
import pandas as pd
from PIL import Image  # Best for resize of image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [53]:
# set the random seed to get constant random numbers
np.random.seed(1000)

In [54]:
# setting keras background to tensorflow
os.environ["KERAS_BACKEND"] = 'tensorflow'

Steps to be followed for CNN

1. Input layer.
2. Convolutional layer,max pooling,normalization,dropout layers.
3. Convolutional layer,max pooling,normalization,dropout layers.
4. Faltten the result.
5. Dense layer,normalization,dropout.
6. Dense layer,normalization,dropout.
7. Output layer with sigmoid activation function.

In [65]:
image_directory = "cells/"

# Define size
SIZE = 64

# Define an empty list
dataset = []

# Create an empty list of labels ==> label 0(parasites),label 1(uninfected)
label = []

# read images
parasited_images = os.listdir(image_directory + 'parasites/')

# iterate through every image
for i, image_name in enumerate(parasited_images):

    # Check if the image is png or not
    if(image_name.split(".")[1] =="png"):

        # Read the image
        image = cv2.imread(image_directory + "parasites/" + image_name)

        # Convert BGR into RGB
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        # convert into pil image for resize
        image = Image.fromarray(image)

        # Resize the image
        image = image.resize((SIZE,SIZE))

        # Append the image data
        dataset.append(np.array(image))

        # Append the label as 0 for parasites
        label.append(0)

# Do for uninfected images
uninfected_images = os.listdir(image_directory + "uninfected/")
for i,input_image in enumerate(uninfected_images):

    # Check the image is png or not
    if(input_image.split(".")[1] =='png'):

        # read the image
        image = cv2.imread(image_directory + "uninfected/" + input_image)

        # Convert BGR into RGB
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        # Convert into PIL format
        image = Image.fromarray(image)

        # Resize the image
        image = image.resize((SIZE,SIZE))

        # Append the image data into dataset
        dataset.append(image)
        
        # Append the labels
        label.append(1)

# Convert dataset and labels into arrays
dataset = np.array(dataset)
label = np.array(label)

In [66]:
# Define input shape
INPUT_SHAPE = (SIZE,SIZE,3)

# Define input
input = keras.layers.Input(shape = INPUT_SHAPE)

# define 1st convolutional layer and pass input into it.
conv1 = keras.layers.Conv2D(filters = 32, kernel_size = (3,3),activation = 'relu', padding = 'same',strides = (1,1))(input)

# Define max pooling layer and pass the 1st convolutional layer into it.
pool1 = keras.layers.MaxPooling2D(pool_size = (2,2))(conv1)

# Define batch normalization and pass maxpooling into it
norm1 = keras.layers.BatchNormalization(axis = -1)(pool1)

# define dropout layer
drop1 = keras.layers.Dropout(rate = 0.2)(norm1)

# In the same way define second layers
conv2 = keras.layers.Conv2D(filters = 32, kernel_size = (3,3),activation = 'relu', padding = 'same', strides = (1,1))(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size = (2,2))(conv2)
norm2 = keras.layers.BatchNormalization(axis = -1)(pool2)
drop2 = keras.layers.Dropout(rate = 0.2)(norm2)

# flatten the matrix
flat = keras.layers.Flatten()(drop2)

# Define hidden layer
hidden1 = keras.layers.Dense(units = 512,activation = 'relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate = 0.2)(norm3)

# Define hidden layer
hidden2 = keras.layers.Dense(units = 256,activation = 'relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate = 0.2)(norm4)

# Define output layers
output = keras.layers.Dense(2,activation = 'sigmoid')(drop4)

model = keras.Model(inputs = input,outputs = output)
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
print(model.summary())


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 64, 64, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 32, 32, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 32, 32, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 16, 16, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,340,130 (16.56 MB)

 Trainable params: 4,338,466 (16.55 MB)

 Non-trainable params: 1,664 (6.50 KB)

None


**Splliting data intro training and testing**

In [58]:
x_train,x_test,y_train,y_test = train_test_split(dataset, to_categorical(np.array(label)),test_size = 0.2,random_state = 0)

**Fit the model**

In [67]:
history = model.fit(np.array(x_train),
                    y_train,
                    batch_size = 32,
                    verbose = 1,
                    epochs = 100,
                    validation_split = 0.1,
                    shuffle = True
                   )

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 142ms/step - accuracy: 0.6208 - loss: 0.9737 - val_accuracy: 0.4167 - val_loss: 28.8960
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.9108 - loss: 0.2805 - val_accuracy: 0.4167 - val_loss: 45.4319
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.9365 - loss: 0.1509 - val_accuracy: 0.4167 - val_loss: 44.5429
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.9837 - loss: 0.0584 - val_accuracy: 0.4167 - val_loss: 31.2362
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.9711 - loss: 0.0999 - val_accuracy: 0.4167 - val_loss: 31.4675
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.9591 - loss: 0.1142 - val_accuracy: 0.4167 - val_loss: 25.3652
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.9932 - loss: 0.0357 - val_accuracy: 0.4167 - val_loss: 19.7463
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.9867 - loss: 0.0403 - val_accuracy: 0.4167 - v

In [64]:
print(f"Test_Accuracy is : {model.evaluate(np.array(x_test),np.array(y_test))[1]*100}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7937 - loss: 1.0829
Test_Accuracy is : 80.0000011920929
